In [19]:
from models.DAX import DAX_quantile_regression, DAX_baseline
from models.energy import energy_quantile_regression
from functions import get_energy,get_DAX,merge_submissions,check_submission
from functions.prepare_data import split_time
from functions.evaluation import evaluate_horizon
from tqdm import tqdm
from functions import evaluate_dax

In [15]:
from functions import naive_ensemble

In [16]:
daxdata = get_DAX.get()

In [17]:
base = DAX_baseline.DAX_baseline(daxdata)

In [18]:
base

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-30,DAX,1 day,-3.321265,-0.546439,0.068631,0.718274,2.719772
1,2023-11-30,DAX,2 day,-4.087895,-0.817404,0.080265,1.000970,3.570492
2,2023-11-30,DAX,5 day,-5.259161,-0.912912,0.164214,1.232426,4.628697
3,2023-11-30,DAX,6 day,-6.282629,-1.044070,0.203116,1.443441,5.367396
4,2023-11-30,DAX,7 day,-6.735742,-1.091646,0.343478,1.594461,6.259934


In [5]:
quantile = DAX_quantile_regression.DAX_quantile_regression(daxdata)

In [6]:
df_dax=naive_ensemble.ensemble(base,quantile,0.5,0.5)

In [7]:
energydata= get_energy.get()

  0%|          | 0/258 [00:00<?, ?it/s]/Users/stephantimpe/PycharmProjects/probabilistic-forecasting-challenge/functions/get_energy.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  energydata = pd.concat([energydata, pd.DataFrame(rawdata, columns=col_names)])
100%|██████████| 258/258 [00:33<00:00,  7.71it/s]


In [8]:
df_energy= energy_quantile_regression.energy_quantile_regression(energydata)

                         QuantReg Regression Results                          
Dep. Variable:                 gesamt   Pseudo R-squared:               0.5070
Model:                       QuantReg   Bandwidth:                      0.6979
Method:                 Least Squares   Sparsity:                        11.98
Date:                Wed, 29 Nov 2023   No. Observations:                43242
Time:                        18:17:11   Df Residuals:                    43206
                                        Df Model:                           35
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         58.7875      0.175    336.028      0.000      58.445      59.130
weekday       -1.8470      0.014   -128.271      0.000      -1.875      -1.819
month_2        0.4379      0.142      3.078      0.002       0.159       0.717
month_3       -3.2381      0.139    -23.318      0.0

/Users/stephantimpe/anaconda3/envs/Forecasting_Challenge_2/lib/python3.9/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (1000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


In [20]:
df_energy

,forecast_date,target,horizon,q0.025,q0.25,q0.5,q0.75,q0.975
0,2023-11-29,energy,36 hour,39.598251,47.270818,51.399704,54.56094,57.74445
1,2023-11-29,energy,40 hour,50.56704,59.961533,64.019735,68.926358,74.359249
2,2023-11-29,energy,44 hour,48.834292,56.250523,59.346171,62.739859,66.775318
3,2023-11-29,energy,60 hour,38.611751,45.395177,49.552746,53.132607,56.992767
4,2023-11-29,energy,64 hour,49.58054,58.085891,62.172777,67.498025,73.607566
5,2023-11-29,energy,68 hour,47.847792,54.374882,57.499213,61.311526,66.023635


In [9]:
df = merge_submissions.merge(df_energy,df_dax)

  forecast_date      target  horizon     q0.025      q0.25       q0.5  \
0    2023-11-29         DAX    1 day  -2.865982  -0.544096   0.065096   
1    2023-11-29         DAX    2 day  -3.221294  -0.620475   0.194377   
2    2023-11-29         DAX    5 day  -4.278427   -1.10553  -0.166646   
3    2023-11-29         DAX    6 day  -4.641268  -1.013727   0.023049   
4    2023-11-29         DAX    7 day  -4.554499  -0.862567   0.270835   
0    2023-11-29      energy  36 hour  39.598251  47.270818  51.399704   
1    2023-11-29      energy  40 hour   50.56704  59.961533  64.019735   
2    2023-11-29      energy  44 hour  48.834292  56.250523  59.346171   
3    2023-11-29      energy  60 hour  38.611751  45.395177  49.552746   
4    2023-11-29      energy  64 hour   49.58054  58.085891  62.172777   
5    2023-11-29      energy  68 hour  47.847792  54.374882  57.499213   
0    2023-11-29  infections   0 week         NA         NA         NA   
1    2023-11-29  infections   1 week         NA    

In [10]:
df.to_csv('../forecasts/20231129_JonSnow.csv',index=False)

OSError: Cannot save file into a non-existent directory: '../forecasts'

In [ ]:
from functions import check_submission

In [13]:
import sys
import pandas as pd
import numpy as np
import warnings
from datetime import datetime

def check_df(df):
    EXPECTED_COLS = ["forecast_date", "target", "horizon", "q0.025", "q0.25",
                    "q0.5", "q0.75", "q0.975"]
    LEN_EXP_COLS = len(EXPECTED_COLS)

    # if exclude_weather == True:
    #     print("Excluding weather variables!")
    #     TARGETS = ["DAX", "energy"]
    # else:
    TARGETS = ["DAX", "energy"]

    TARGET_VALS = dict(DAX = [str(i) + " day" for i in (1,2,5,6,7)],
                    energy = [str(i) + " hour" for i in (36,40,44,60,64,68)],
                    infections = [str(i) + " week" for i in (0,1,2,3,4)])

    TARGET_LEN = dict(DAX = len(TARGET_VALS["DAX"]),
                    energy = len(TARGET_VALS["energy"]),
                    infections = len(TARGET_VALS["infections"])
                    )

    TARGET_PLAUS = dict(DAX = [-20, 20],
                        energy = [0,250],
                        infections = [0,9000])

    COLS_QUANTILES = ["q0.025", "q0.25", "q0.5", "q0.75", "q0.975"]

    print("Start checking...")
    print("---------------------------")
    col_names = df.columns


    print("Checking the Columns...")
    # Check column length
    if len(col_names) != LEN_EXP_COLS:
        print("Dataset contains ",len(col_names), "columns. Required are",LEN_EXP_COLS)
        print("Stopping early...")
        sys.exit()

    if set(col_names) != set(EXPECTED_COLS):
        print("Dataset does not contain the required columns (or more).")
        missing_cols = set(EXPECTED_COLS) - set(col_names)
        print("The missing columns are:", missing_cols)
        print("Stopping early...")
        sys.exit()

    for i,col in enumerate(EXPECTED_COLS):
        if col == col_names[i]:
            continue
        else:
            print("Columns not in correct order. Order should be:", EXPECTED_COLS)
            print("Your order is:", col_names.values)
            print("Stopping early...")
            sys.exit()

    # Date Col
    print("Checking type of columns...")
    try:
        df["forecast_date"] = pd.to_datetime(df["forecast_date"], format="%Y-%m-%d",
                                            errors="raise")
    except (pd.errors.ParserError, ValueError):
        print("Could not parse Date in format YYYY-MM-DD")
        print("Stopping early...")
        sys.exit()

    try:
        df["target"] = df["target"].astype("object", errors="raise")
    except ValueError:
        print("Cannot convert target column to String.")
        print("Stopping early...")
        sys.exit()

    try:
        df["horizon"] = df["horizon"].astype("object", errors="raise")
    except ValueError:
        print("Cannot convert horizon column to String.")
        print("Stopping early...")
        sys.exit()

    for cq in COLS_QUANTILES:
        if pd.to_numeric(df[cq], errors="coerce").isna().any():
            print("----WARNING: Some elements in",cq,"column are not numeric. This may be fine if you only submit 2 out of 3 targets.")
            print("")
            # print("Stopping early...")
            # sys.exit()

    print("Checking if the Dates make sense...")

    if len(pd.unique(df["forecast_date"])) > 1:
        print("forecast_date needs to be the same in all rows.")
        print("Stopping early...")
        sys.exit()

    if df["forecast_date"][0].date() < datetime.today().date():
        print("----WARNING: Forecast date should not be in the past.")
        print("")
        # warnings.warn("Forecast date should not be in the past.")

    if df["forecast_date"][0].weekday() != 2:
        print("----WARNING: Forecast date should be a Wednesday.")
        print("")
        # warnings.warn("Forecast date should be a Wednesday")

    print("Checking targets...")

    if not df["target"].isin(TARGETS).all():
        print(f"Target column can only contain {TARGETS}. Check spelling.")
        print("Stopping early...")
        sys.exit()

    for target in TARGETS:

        if len(df[df["target"] == target]) != TARGET_LEN[target]:
            if target == "demand":
                print("Exactly 6 rows need to have target = ", target)
            else:
                print("Exactly 5 rows need to have target =", target)
            print("Stopping early...")
            sys.exit()

        if (df[df["target"] == target]["horizon"] != TARGET_VALS[target]).any():
            print("Target", target, "horizons need to be (in this order):", TARGET_VALS[target])
            print("Stopping early...")
            sys.exit()

        if (df[df["target"] == target][COLS_QUANTILES] < TARGET_PLAUS[target][0]).any(axis=None) or \
            (df[df["target"] == target][COLS_QUANTILES] > TARGET_PLAUS[target][1]).any(axis=None):
            print("----WARNING: Implausible values for",target,"detected. You may want to re-check.")
            print("")
            # warnings.warn("Implausible values for "+str(target)+" detected. You may want to re-check them.")

    print("Checking quantiles...")

    ALL_NAN_IDX = df[df.isna().any(axis=1)].index
    NAN_TARGET_IDX_LIST = []

    if len(ALL_NAN_IDX) != 0:
        NAN_TARGET = df.iloc[ALL_NAN_IDX[0]]["target"]
        NAN_TARGET_LENS = dict(DAX = 5,
                            energy = 6,
                            infections = 5)

        NAN_TARGET_IDX_LIST = df[df["target"] == NAN_TARGET].index

        print("Assume that --",NAN_TARGET,"-- is your NaN-target. Please DOUBLECHECK if this is correct.")

        if len(ALL_NAN_IDX) > NAN_TARGET_LENS[NAN_TARGET]:
            print("Your dataframe contains more NaNs than entries for target",NAN_TARGET,".")
            print("Stopping early...")
            sys.exit()
    else:
        print("Seems like you submitted all three targets. Good job!")

    for i, row in df.iterrows():
        if i in NAN_TARGET_IDX_LIST:
            continue

        diffs = row[COLS_QUANTILES].diff()
        if diffs[1:].isna().any():
            print("Something is wrong with your quantiles.")
            print("Stopping early...")
            sys.exit()
        diffs[0] = 0
        if (diffs < 0).any():
            print("Predictive quantiles in row",i,"are not ordered correctly (need to be non-decreasing)")
            print("Stopping early...")
            sys.exit()

    print("---------------------------")
    print("Looks good!")
    
check_df(df)

Start checking...
---------------------------
Checking the Columns...
Checking type of columns...
----WARNING: Some elements in q0.025 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.25 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.5 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.75 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

----WARNING: Some elements in q0.975 column are not numeric. This may be fine if you only submit 2 out of 3 targets.

Checking if the Dates make sense...


AttributeError: 'Series' object has no attribute 'date'

In [ ]:
df